In [1]:
from pyspark import SparkContext


for k, v in spark.sparkContext.getConf().getAll():
    print(f"{k} = {v}")


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
17,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Cannot run multiple SparkContexts at once; existing SparkContext(app=livy-session-17, master=local) created by __init__ at /tmp/7633275838383001008:600 
Traceback (most recent call last):
  File "/home/glue_user/spark/python/pyspark/context.py", line 195, in __init__
    SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
  File "/home/glue_user/spark/python/pyspark/context.py", line 438, in _ensure_initialized
    raise ValueError(
ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=livy-session-17, master=local) created by __init__ at /tmp/7633275838383001008:600 



In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaAvroConsumerNotebook") \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
kafka_url = "broker:29092"
schema_registry_url = "https://rnung-103-224-144-138.a.free.pinggy.link"
kafka_producer_topic = "topic_prefix.kafkaDB.users"
schema_registry_subject = f"{kafka_producer_topic}-value"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
import os

path = "/home/glue_user/spark/jars/commons-pool2-2.11.1.jar"
if os.path.exists(path):
    print("✅ JAR file exists.")
else:
    print("❌ JAR file not found.")


# from pyspark.sql.avro.functions import from_avro

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
20,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

✅ JAR file exists.

In [5]:
import pyspark.sql.functions as func
spark.sparkContext.setLogLevel("ERROR")


wikimedia_df = spark \
.readStream\
.format("kafka")\
.option("kafka.bootstrap.servers", kafka_url)\
.option("subscribe", kafka_producer_topic)\
.option("startingOffsets", "earliest")\
.option("group.id", "aman103")\
.load()
wikimedia_df.printSchema()




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

In [6]:
# wikimedia_df.printSchema()
wikimedia_df = wikimedia_df.withColumn("value_str", func.col("value").cast("string"))

#  get magic byte value
wikimedia_df = wikimedia_df.withColumn("magicByte", func.expr("substring(value, 1, 1)"))

#  get schema id from value
wikimedia_df = wikimedia_df.withColumn("valueSchemaId", func.expr("substring(value, 2, 4)"))

# remove first 5 bytes from value
wikimedia_df = wikimedia_df.withColumn("fixedValue", func.expr("substring(value, 6, length(value)-5)"))

# creating a new df with magicBytes, valueSchemaId & fixedValue
wikimedia_value_df = wikimedia_df.select("magicByte", "valueSchemaId", "fixedValue")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# write to sink(console)
## trigger is used for batch interval
wikimedia_value_df \
.writeStream \
.format("console") \
.outputMode("append") \
.option("truncate", "true") \
.start() \
.awaitTermination(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Query [id = 1099fb92-5f5a-41e4-ba4e-db0b3f6ee3c8, runId = 30af5d75-a9d8-4863-980f-f394a6490231] terminated with exception: Writing job aborted
Traceback (most recent call last):
  File "/home/glue_user/spark/python/pyspark/sql/streaming.py", line 105, in awaitTermination
    return self._jsq.awaitTermination(int(timeout * 1000))
  File "/home/glue_user/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/home/glue_user/spark/python/pyspark/sql/utils.py", line 196, in deco
    raise converted from None
pyspark.sql.utils.StreamingQueryException: Query [id = 1099fb92-5f5a-41e4-ba4e-db0b3f6ee3c8, runId = 30af5d75-a9d8-4863-980f-f394a6490231] terminated with exception: Writing job aborted

